<a href="https://colab.research.google.com/github/Ujjwalpaliwal/FreeCodeCamp_Project/blob/main/Book_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-06-30 08:46:56--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  81.7MB/s    in 0.3s    

2025-06-30 08:46:57 (81.7 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
df_ratings

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0
...,...,...,...
1149775,276704,1563526298,9.0
1149776,276706,0679447156,0.0
1149777,276709,0515107662,10.0
1149778,276721,0590442449,10.0


In [8]:
user_ratings = df_ratings.groupby('user', as_index = False)['rating'].count().rename(columns = { 'rating':'usercount'})
book_ratings = df_ratings.groupby('isbn', as_index = False)['rating'].count().rename(columns = {'rating':'titlecount'})
df_ratings = pd.merge(left = df_ratings, right = user_ratings, on = 'user')
df_ratings = pd.merge(left = df_ratings, right = book_ratings, on = 'isbn')

df = df_ratings.loc[(df_ratings['usercount'] >= 200) & (df_ratings['titlecount'] >= 100)]
df = df.drop(['usercount', 'titlecount'], axis = 1)
df = pd.merge(left = df, right = df_books, on = 'isbn')

In [9]:
df = df.drop_duplicates(['user', 'title'])
df_pivot = pd.pivot(df, values = 'rating', index = 'title', columns = 'user' )
df_pivot = df_pivot.fillna(0)

In [10]:
import scipy as spy
df_matrix = spy.sparse.csr_matrix(df_pivot.values)

In [11]:
model = NearestNeighbors(algorithm = 'brute', metric = 'cosine')
model.fit(df_matrix)


NearestNeighbors(algorithm='brute', metric='cosine')

In [12]:
def get_recommends(book = ""):
  x = df_pivot[df_pivot.index == book]
  dist, ind = model.kneighbors(x, n_neighbors = 6)
  recommended_books = []
  reco_books = []
  dist = dist.flatten()
  ind = ind.flatten()
  for i in range(len(ind)):
      if i == 0:
        recommended_books.append(df_pivot.index[ind[i]])
      else:
        reco_book = df_pivot.index[ind[i]]
        reco_dist = dist[i]
        reco_books.append([reco_book, reco_dist])
  #reversing the list order because according to forums, the test function has a bug
  reco_books = reco_books[::-1]
  recommended_books.append(reco_books)



  return recommended_books

In [13]:
ans = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(ans)

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", np.float32(0.8016211)], ['The Weight of Water', np.float32(0.77085835)], ['The Surgeon', np.float32(0.7699411)], ['I Know This Much Is True', np.float32(0.7677075)], ['The Lovely Bones: A Novel', np.float32(0.7234864)]]]


In [14]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", np.float32(0.8016211)], ['The Weight of Water', np.float32(0.77085835)], ['The Surgeon', np.float32(0.7699411)], ['I Know This Much Is True', np.float32(0.7677075)], ['The Lovely Bones: A Novel', np.float32(0.7234864)]]]
You passed the challenge! 🎉🎉🎉🎉🎉
